Перед початком роботи над etl, варто дослідити саму структуру датасету

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import sqlite3
from sqlalchemy import create_engine


df = pd.read_csv('../data/healthcare-dataset-stroke-data.csv')  # read CSV

Отож ми маємо датасет з 11 колонками: gender - стать, age - вік, hypertension - чи страждає пацієнт гіпертонією, heart_disease - чи маєме пацієнт захворювання серця, ever_married - чи був коли-небудь пацієнт одруженим, work_type - тип роботи, Residence_type - де проживає пацієнт, avg_glucose_level - середній рівень цекру в крові, bmi - індекс маси тіла, smoking_status - статус куріння, stroke - чи стикався пацієн з інсультом. Три з цих характеристик чисельні, п'ять категоріальні, та чотири булеві (у датасеті вони відображені в вигляді 0 та 1)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Подивимось на категоріальні ознаки, які ми маємо

In [3]:
df['gender'].value_counts()

Female    2994
Male      2115
Other        1
Name: gender, dtype: int64

Значення Other зустрічається всього раз, тому напевно варто його видалити

In [4]:
df.drop(labels=3116, inplace=True, axis=0)  # gender 'other' delete

In [5]:
df['ever_married'].value_counts()

Yes    3353
No     1756
Name: ever_married, dtype: int64

In [6]:
df['work_type'].value_counts()

Private          2924
Self-employed     819
children          687
Govt_job          657
Never_worked       22
Name: work_type, dtype: int64

In [7]:
df['Residence_type'].value_counts()

Urban    2596
Rural    2513
Name: Residence_type, dtype: int64

In [8]:
df['smoking_status'].value_counts()

never smoked       1892
Unknown            1544
formerly smoked     884
smokes              789
Name: smoking_status, dtype: int64

Статус куріння 'Unknown' зустрічається вибірці аж 1544, це близько 30% усієї вибірки, тож ми змушені видалити цю властивість

In [9]:
df = df.drop('smoking_status', axis=1)

Знайдемо усі унікальні значення

In [11]:
gen = pd.Series(pd.unique(df['gender'])).rename('Gender')
mar = pd.Series(pd.unique(df['ever_married'])).rename('Married')
res = pd.Series(pd.unique(df['Residence_type'])).rename('Residence_type')
work = pd.Series(pd.unique(df['work_type'])).rename('Work')

In [12]:
def change_data(ser, df):
    for i in range(0, ser.shape[0]):
        df = df.replace(ser[i], i)
    return df

In [13]:
df['gender'] = change_data(gen, df['gender'])
df['ever_married'] = change_data(mar, df['ever_married'])
df['Residence_type'] = change_data(res, df['Residence_type'])
df['work_type'] = change_data(work, df['work_type'])

In [17]:
df.isna().any()

id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                   True
stroke               False
dtype: bool

In [21]:
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'avg_glucose_level', 'bmi', 'stroke']]

imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(df)
df_imputed = pd.DataFrame(imputed, columns=df.columns)


In [23]:
df_imputed.isna().any()

id                   False
gender               False
age                  False
hypertension         False
heart_disease        False
ever_married         False
work_type            False
Residence_type       False
avg_glucose_level    False
bmi                  False
stroke               False
dtype: bool

І скрипти внесення оброблених даних до сховища

In [3]:
user = 'root'
password = 'secret'
host = 'localhost'
port = 3306
database = 'cw'

In [ ]:
def get_connection():
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

def fill_db(data, table):
    data.to_sql(table, con=conn, if_exists='append', index=False)

def get_connection_sqlite(db):
        conn = sqlite3.connect(db)
        return conn

In [ ]:
try:
    conn = get_connection()
    print(
        f"Connection to the {host} for user {user} created successfully.")
except Exception as ex:
    print("Connection could not be made due to the following error: \n", ex)

    # --Gen
fill_db(gen, 'gen')

    # --Mar
fill_db(mar, 'mar')

    # --Res
fill_db(res, 'res')

    # --Work
fill_db(work, 'work')

X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type',
            'avg_glucose_level', 'bmi', 'stroke']]

fill_db(df_imputed, 'stroke_prediction_dataset')